# Model evaluation

In [1]:
import sys
sys.path.append('../')
from mocatml.utils import *
from tsai.basics import *
import wandb

In [2]:
my_setup()

os              : Linux-6.2.0-37-generic-x86_64-with-glibc2.31
python          : 3.9.17
tsai            : 0.3.8
fastai          : 2.7.13
fastcore        : 1.5.29
torch           : 2.1.1+cu121
device          : cpu
cpu cores       : 4
threads per cpu : 2
RAM             : 15.3 GB
GPU memory      : N/A


In [3]:
config = yaml2dict('./config/convgru/eval.yaml')
config

```json
{ 'ddata': { 'path': '~/arclab_shared/mc_density_data/TLE_density_all_x15x15.mat',
             'valid_split': True},
  'device': 'cpu',
  'learner': { 'fname': 'learner.pkl',
               'path': 'vrodriguezf90/mocatml/density-forecaster:v1',
               'wandb': { 'dir': '~/Github/mocat-ml',
                          'download_path': 'wandb/artifacts/density-forecaster',
                          'enabled': True}},
  'mmap': True}
```

In [4]:
# Set device
default_device(0 if config.device == 'cpu' else config.device)

device(type='cpu')

In [5]:
wandb_api = wandb.Api()
if config.learner.wandb.enabled:
    aux = wandb_api.artifact(config.learner.path)
    lp = aux.download(root= Path(config.learner.wandb.dir).expanduser()/\
                            Path(config.learner.wandb.download_path))
    lp = lp/Path(config.learner.fname)
else:
    lp = Path(config.learner.path)/Path(config.learner.fname)
learn = load_learner(lp, cpu=default_device() == torch.device("cpu"))
learn

wandb: WARNING Path /workspaces/mocat-ml /wandb/ wasn't writable, using system temp directory
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Path /workspaces/mocat-ml /wandb/ wasn't writable, using system temp directory.
wandb: WARNING Path /workspaces/mocat-ml /wandb/ wasn't writable, using system temp directory
wandb: Downloading large artifact density-forecaster:v1, 490.61MB. 4 files... 
wandb:   4 of 4 files downloaded.  
Done. 0:0:33.5
wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


In [ ]:
X = data = np.load(Path(config.data.path).expanduser(), 
               mmap_mode='c' if config.mmap else None)